In [ ]:
## importing the required packages
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.corpus import words
import nltk.sentiment.sentiment_analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from spellchecker import SpellChecker
import csv
import time
from textblob import TextBlob
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import requests

In [ ]:
## reading th CSV that was exported with the tweets; the filename depends on the number of tweets
train_tweets = pd.read_csv(r'C:\Users\JMSch\Documents\MEGA\05_Paris\ESCP Business School\MSc Big Data & Business Analytics\Research Methods II\Sentiment analysis\eolienne-2k-tweets.csv', dtype=str)


train_tweets

In [ ]:
## STEP I: TRANSLATION
## https://stackabuse.com/text-translation-with-google-translate-api-in-python/
## Translating the tweets to English and saving the translations as a new column
translation = []

NumberOfIterations = train_tweets.count  
start = time.time() 

for i, row in train_tweets.iterrows():
    #try:
        translator = Translator()
        ## print initial tweet value
        tweet = str(row['Text'])
        print(tweet)
        tweet_en = translator.translate(tweet)
        ## print the english translation of the tweet
        print (tweet_en.text)
        ## add the translation to a list
        translation.append(tweet_en.text)
        PercentageFinished = (i/NumberOfIterations)*100
        end = time.time() 
        elapsed = end - start
        TimeToGo = elapsed/i*(NumberOfIterations)
        print(f"{TimeToGo} seconds remaining... ({PercentageFinished}% complete)")
    #except:
        translation.append('error')
        #pass

## append list as a column to the dataframe
train_tweets['Translations'] = translation


In [ ]:
train_tweets

In [ ]:
## STEP II: TEXT MANIPULATION

## Text manipulation; see further comments --> make a decision about removing short words
print(train_tweets['Translations'])

## Remove unwanted text patterns from the tweets.
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt) 
    return input_txt

##  Removing Twitter Handles (@user)
train_tweets['Tidy Text'] = np.vectorize(remove_pattern)(train_tweets['Translations'], "@[\w]*") 
##  Removing Punctuations, Numbers, and Special Characters
train_tweets['Tidy Text'] = train_tweets['Tidy Text'].str.replace("[^a-zA-Z#]", " ")

print(train_tweets['Tidy Text'])


In [ ]:
train_tweets['Translations']


In [ ]:
## STEP IIIA: TOKENISATION

## Tokenisation of words, and removing stopwords from the translated tweet contect
token_transl = []

for i, row in train_tweets.iterrows():
    if str(row['Tidy Text']) != "":
        #try:
            ## row['Translation'] --> select one column FROM the ROW 
            sentence = str(row['Tidy Text'])
            stop_words = set(stopwords.words('english'))  
            word_tokens = word_tokenize(sentence) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words] 
            print(filtered_sentence) 
            token_transl.append(filtered_sentence)
            #print(filtered_sentence)      
        #except:
        #    pass

train_tweets['Tokenised Translation'] = token_transl

In [ ]:
## STEP IIIB: SPELL CORRECTION

## Text manipulation; see further comments --> make a decision about removing short words
print(train_tweets['Tokenised Translation'])

spell = SpellChecker()

for i, row in train_tweets.iterrows():
    try:
        ## print initial tweet value
        tweet = str(row['Tokenised Translation'])
        print(tweet)
# find those words that may be misspelled
        misspelled = spell.unknown(tweet)
        for word in misspelled:
            # Get the one `most likely` answer
            print(spell.correction(word))

            # Get a list of `likely` options
            print(spell.candidates(word))
    except:
        pass

In [ ]:
## STEP IV: SENTIMENT CALCULATION

sentiment_score = []

for i, row in train_tweets.iterrows():
    try:
        print(row['Tokenised Translation'])
        text = str(row['Tokenised Translation'])
        testimonial = TextBlob(text)
        testimonial.sentiment
        Score = testimonial.sentiment.polarity
        print (Score)
        sentiment_score.append(Score)
    except:
        sentiment_score.append('error')
        pass

train_tweets['Sentiment Score'] = sentiment_score

In [ ]:
train_tweets.to_csv(r'C:\Users\JMSch\Documents\MEGA\05_Paris\ESCP Business School\MSc Big Data & Business Analytics\Research Methods II\Sentiment Analysis Twitter.csv', index = False)

In [ ]:
## STEP IV: SENTIMENT CALCULATION (USING DIFFERENT PACKAGE)

sid = SentimentIntensityAnalyzer()

for i, row in train_tweets.iterrows():
    print(row['Tokenised Translation'])
    sentence = str(row['Tokenised Translation'])
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()
